# 第10章: ベクトル空間法 (II)
第10章では，前章に引き続き単語ベクトルの学習に取り組む．

## 90. word2vecによる学習
81で作成したコーパスに対してword2vecを適用し，単語ベクトルを学習せよ．さらに，学習した単語ベクトルの形式を変換し，86-89のプログラムを動かせ．

In [1]:
!head -n 5 ../../chapter09/work/enwiki-20150112-400-r100-10576-compound_replaced.txt

Anarchism
Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system
As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categories of

In [2]:
from gensim.models.word2vec import LineSentence, Word2Vec
from gensim.test.utils import datapath

sentences = LineSentence('../../chapter09/work/enwiki-20150112-400-r100-10576-compound_replaced.txt')
model = Word2Vec(sentences, size=100, window=5, min_count=10, sg=1, hs=0, negative=5, workers=4)
model.save('../work/word2vec_d100_w5_sg.model')

In [3]:
from gensim.models.word2vec import Word2Vec

def knock_90():
    model = Word2Vec.load("../work/word2vec_d100_w5_sg.model")
    print("United_States の単語ベクトル:")
    print(model.wv["United_States"])
    
    print("\nUnited_States と U.S の類似度:")
    print(model.wv.similarity("United_States", "U.S"))

    print("\nEngland と類似度の高い単語10件")
    print(model.wv.most_similar(positive=['England'], topn=10))
    
    print("\nSpain - Madrid + Athens と類似度の高い単語10件")
    print(model.wv.most_similar(positive=["Spain", "Athens"], negative=["Madrid"], topn=10))
    
knock_90()

United_States の単語ベクトル:
[-0.28676808 -0.33307534 -0.34927177  0.3465644   0.04432601 -0.4322733
  0.371253   -0.8625143   0.21403056  0.33887684  0.24313676 -0.01577393
 -0.77665657  0.3739606  -0.24772237 -0.16604081 -0.13625292  0.12585013
 -0.42808485 -0.05091853 -0.7095111  -0.4920803  -0.12442167  0.4092078
  0.15587461  0.1395456   0.22662784  0.06617327  0.02785342  0.4088276
  0.20289661  0.37518272 -0.29654968  0.46529588 -0.05565928 -0.36331254
  0.18550792 -0.33341426 -0.19583142  0.16085361 -0.28075716  0.3405469
  0.3248612  -0.00649171  0.4347056  -0.05338414  0.12131537  0.03553332
 -0.14288971  0.3956565  -0.05746122 -0.3386314   0.07332081 -0.29849255
  0.5082681   0.30424362  0.34834433  0.2770656  -0.43447274 -0.03748158
  0.42596143 -0.26388896  0.53265697  0.17335704 -0.5438138   0.44658875
  0.24118969 -0.17824431  0.09972241 -0.19586906 -0.03400543  0.20292345
 -0.10313662 -0.11593749 -0.13695014  0.17198503 -0.00220675 -0.38764527
  0.20446423 -0.105957    0.0373

## 91. アナロジーデータの準備
単語アナロジーの評価データをダウンロードせよ．このデータ中で": "で始まる行はセクション名を表す．例えば，": capital-common-countries"という行は，"capital-common-countries"というセクションの開始を表している．ダウンロードした評価データの中で，"family"というセクションに含まれる評価事例を抜き出してファイルに保存せよ．

In [4]:
def knock_91():
    with open("../data/questions-words.txt") as f_in, open("../work/questions-words-family.txt", "w") as f_out:
        is_family = False
        for line in f_in:
            if line.startswith(": family"):
                is_family = True
                continue
            if line.startswith(": ") and is_family:
                break
            if is_family:
                f_out.write(line)

knock_91()

## 92. アナロジーデータへの適用
91で作成した評価データの各事例に対して，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

In [ ]:
import pickle
from functools import lru_cache
from gensim.models.word2vec import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

with open('../../chapter09/work/word_vector.pkl', 'rb') as f:
    word_vectors_85 = pickle.load(f)
with open('../../chapter09/work/word_to_index.pkl', 'rb') as f:
    word_to_index_85 = pickle.load(f)

model_90 = Word2Vec.load("../work/word2vec_d100_w5_sg.model")
    
def get_similar_word_and_vector_knock_85(base_word, minus_word, plus_word):
    
    @lru_cache(maxsize=100)
    def get_word_vector(word):
        return word_vectors_85[word_to_index_85[word.lower()]]
    
    base_word_vector = get_word_vector(base_word.lower())
    minus_word_vector = get_word_vector(minus_word.lower())
    plus_word_vector = get_word_vector(plus_word.lower())
    target_vector = base_word_vector - minus_word_vector + plus_word_vector
    cosine_similarities = { word:cosine_similarity([target_vector],[word_vectors_85[index]])[0][0] for word,index in word_to_index_85.items()}
    return sorted(cosine_similarities.items(), key=lambda x:x[1], reverse=True)[0]

def get_similar_word_and_vector_knock_90(base_word, minus_word, plus_word):
    try:
        return model_90.wv.most_similar(positive=[base_word, plus_word], negative=[minus_word], topn=1)[0]
    except KeyError:
        return "UNK", 0.0

def knock_92():
    with open("../work/questions-words-family.txt") as f_in, open("../work/questions-words-family-85.txt", "w") as f_out_85, open("../work/questions-words-family-90.txt", "w") as f_out_90:
        for line in f_in:
            minus, base, plus, answer = line.rstrip().split()
            pred_word_85, pred_similarity_85 = get_similar_word_and_vector_knock_85(base, minus, plus)
            pred_word_90, pred_similarity_90 = get_similar_word_and_vector_knock_90(base, minus, plus)
            f_out_85.write(" ".join([minus, base, plus, answer, pred_word_85, str(pred_similarity_85)]).rstrip() + "\n")
            f_out_90.write(" ".join([minus, base, plus, answer, pred_word_90, str(pred_similarity_90)]).rstrip() + "\n")
            
knock_92()

In [ ]:
!head -n 5 ../work/questions-words-family-85.txt

In [ ]:
!head -n 5 ../work/questions-words-family-90.txt

## 93. アナロジータスクの正解率の計算
92で作ったデータを用い，各モデルのアナロジータスクの正解率を求めよ．

In [ ]:
import pandas as pd

def calc_accuracy(file_path):
    df = pd.read_csv(file_path, names=("minus", "base", "plus", "answer", "pred", "similarity"), delim_whitespace=True)
    answer_list = list(df["answer"])
    pred_list = list(df["pred"])
    return sum(1 for pred, answer in zip(pred_list,answer_list) if pred == answer) / float(len(pred_list))
    
def konck_93():
    print("正解率 (Word2Vec): ") + str(calc_accuracy("../work/questions-words-family-90.txt"))
    print("正解率 (PMI)": ) + str(calc_accuracy("../work/questions-words-family-85.txt"))
    
knock_93()

## 94. WordSimilarity-353での類似度計算
The WordSimilarity-353 Test Collectionの評価データを入力とし，1列目と2列目の単語の類似度を計算し，各行の末尾に類似度の値を追加するプログラムを作成せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

## 95. WordSimilarity-353での評価
94で作ったデータを用い，各モデルが出力する類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ．

## 96. 国名に関するベクトルの抽出
word2vecの学習結果から，国名に関するベクトルのみを抜き出せ．

## 97. k-meansクラスタリング
96の単語ベクトルに対して，k-meansクラスタリングをクラスタ数k=5として実行せよ．

## 98. Ward法によるクラスタリング
96の単語ベクトルに対して，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

## 99. t-SNEによる可視化
96の単語ベクトルに対して，ベクトル空間をt-SNEで可視化せよ．